In [40]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np
import random
from torch.utils.data import TensorDataset, DataLoader

In [41]:
# Get images
from PIL import Image
import torchvision.transforms as transforms

# Load the image using PIL
image_path = 'woman.jpg'
#image = Image.open(image_path)

# Convert PIL Image to PyTorch tensor
image_pil = Image.open(image_path).convert('RGB')

image = np.array(image_pil)



In [53]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)
X_original=X
Y_original=Y
#Conversion to Tensors
#X = torch.from_numpy(X).float()
#Y = torch.from_numpy(Y).float()

X= torch.tensor(X, dtype=torch.float32).permute(0, 3, 1, 2) 
Y= torch.tensor(Y, dtype=torch.float32).permute(0, 3, 1, 2) 



In [54]:
import torch.nn as nn
import torch.nn.functional as F

class CNNTranslationModel(nn.Module):
    def __init__(self):
        super(CNNTranslationModel, self).__init__()

        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(8, 8, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1)
        self.conv5 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=1)

        self.upsample1 = nn.Upsample(scale_factor=2)
        self.conv7 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.upsample2 = nn.Upsample(scale_factor=2)
        self.conv8 = nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1)
        self.upsample3 = nn.Upsample(scale_factor=2)
        self.conv9 = nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1)
        self.conv10 = nn.Conv2d(8, 2, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))

        x = self.upsample1(x)
        x = F.relu(self.conv7(x))
        x = self.upsample2(x)
        x = F.relu(self.conv8(x))
        x = self.upsample3(x)
        x = F.relu(self.conv9(x))
        x = F.tanh(self.conv10(x))

        return x

# Assuming the input shape is (batch_size, channels, height, width)
model = CNNTranslationModel()


In [57]:
# Define optimizer and loss function
optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9)
criterion = nn.MSELoss()

# Convert NumPy arrays to PyTorch tensors
#X_tensor = torch.from_numpy(X).float()
#Y_tensor = torch.from_numpy(Y).float()

# Training
for epoch in range(1000):
    # Forward pass
    output = model(X)
    
    # Compute the loss
    loss = criterion(output, Y)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every 100 epochs
    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}/{1000}, Loss: {loss.item()}')

        
        
        
        
        
# Optionally, move the tensors to GPU if available
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#X = X.to(device)
#Y = Y.to(device)

# Create a DataLoader for batch training
#dataset = TensorDataset(X, Y)
#dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Instantiate the model
#model = CNNTranslationModel()

# Define the loss function and optimizer
#criterion = nn.MSELoss()
#optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9)

# Training loop
#num_epochs = 1000
#for epoch in range(num_epochs):
#    total_loss = 0.0
#    for inputs, targets in dataloader:
#        inputs, targets = inputs.to(device), targets.to(device)
#        optimizer.zero_grad()
#        outputs = model(inputs)
#        loss = criterion(outputs, targets)
#        loss.backward()
#        optimizer.step()

#        total_loss += loss.item()

#    average_loss = total_loss / len(dataloader)
#    if(epoch%100==0):
#        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}")


Epoch 1/1000, Loss: 0.0006113707786425948
Epoch 101/1000, Loss: 0.0004401738988235593
Epoch 201/1000, Loss: 0.00032931281020864844
Epoch 301/1000, Loss: 0.0005954992957413197
Epoch 401/1000, Loss: 0.00046015673433430493
Epoch 501/1000, Loss: 0.0005111315986141562
Epoch 601/1000, Loss: 0.00035566691076382995
Epoch 701/1000, Loss: 0.00023971476184669882
Epoch 801/1000, Loss: 0.0002944944426417351
Epoch 901/1000, Loss: 0.000226885182200931


In [58]:
# Evaluation
with torch.no_grad():
    # Convert the model to evaluation mode
    model.eval()
    # Prediction
    output_tensor = model(X)
    output_np = output_tensor.detach().numpy()

    
    # Evaluate the model on the entire dataset
    eval_loss = criterion(model(X), Y)
    print(f'Evaluation Loss: {eval_loss.item()}')


# Post-process the output
output_np *= 128
cur = np.zeros((400, 400, 3))
cur[:, :, 0] = X_original[0][:, :, 0]
cur[:, :, 1:] = output_np[0].transpose(1,2,0)

#cur = (cur * 255).astype(np.uint8)


output_rgb = (lab2rgb(cur) * 255).astype(np.uint8)

input_rgb = (rgb2gray(lab2rgb(cur)) * 255).astype(np.uint8)

imsave("img_result.png", output_rgb)
imsave("img_gray_version.png", input_rgb)


Evaluation Loss: 0.00027899318956770003
